In [1]:
# !pip install mediapipe

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import time

In [3]:

class WorkoutDetector():
    def __init__(self,  static_image_mode=False, model_complexity=1, smooth_landmarks=True,min_detection_confidence=0.5,
    min_tracking_confidence=0.5):
        
        self.static_image_mode = static_image_mode
        self.model_complexity = model_complexity
        self.smooth_landmarks = smooth_landmarks
        self.min_detection_confidence = min_detection_confidence 
        self.min_tracking_confidence = min_tracking_confidence
        
        self.mpDraw =  mp.solutions.drawing_utils
        self.mpPose =  mp.solutions.pose
        self.pose_landmark_map = mp.solutions.pose.PoseLandmark

        self.pose =  self.mpPose.Pose(
            self.static_image_mode,
            self.model_complexity,
            self.smooth_landmarks,
            self.min_detection_confidence,
            self.min_tracking_confidence,
        )


            
    def body_in_frame(self,landmarks,top_landmark_name='nose', botton_landmark_name='right_hip'):
        top_landmark = self.check_lm_visibility(landmarks, top_landmark_name)
        botton_landmark = self.check_lm_visibility(landmarks, botton_landmark_name)
        
        return top_landmark and  botton_landmark
        
    def get_distance_btw_landmarks(self, image, left_lm_a, right_lm_b, draw=True):
        if len(self.lmList) < 33:
            return 0
        lm1, lm1_x , lm1_y = self.lmList[left_lm_a]
        lm2, lm2_x , lm2_y = self.lmList[right_lm_b]
        
        if draw:
            cv2.line(image, (lm1_x, lm1_y), (lm2_x, lm2_y), (255,255,255),3)
            cv2.circle(image, (lm1_x, lm1_y), 15, (23,0,250), cv2.FILLED)
            cv2.circle(image, (lm2_x, lm2_y), 15, (23,0,250), cv2.FILLED)
            
        return lm1_x - lm2_x
            
            
    def check_lm_visibility(self, landmarks, landmark_name):
        landmark_name = landmark_name.upper()
        visibility = landmarks[self.pose_landmark_map[landmark_name].value].visibility  * 100
        return visibility > 90  

    def calculate_angle(self, image,land_marks, draw=True):
        print("LENGHT", len(self.lmList))
        if len(self.lmList) < 33:
            return 0
        pos_a, pos_b, pos_c = land_marks
        lm1, lm1_x , lm1_y = self.lmList[pos_a]
        lm2, lm2_x , lm2_y = self.lmList[pos_b]
        lm3, lm3_x , lm3_y = self.lmList[pos_c]
        radians = np.arctan2(lm3_y - lm2_y, lm3_x - lm2_x) - np.arctan2(lm1_y - lm2_y, lm1_x - lm2_x)
        angle = np.abs(radians * 180.0/np.pi)
        
        if angle > 180.0:
            angle = 360 - angle
        

#         print(angle)    
        if(draw):
#             cv2.putText(image, str(int(angle)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0,0), 3)
            cv2.line(image, (lm1_x, lm1_y), (lm2_x, lm2_y), (255,255,255),3)
            cv2.line(image, (lm2_x, lm2_y), (lm3_x, lm3_y), (255,255,255), 3)
            
            cv2.circle(image, (lm1_x, lm1_y), 15, (23,0,250), cv2.FILLED)
            cv2.circle(image, (lm2_x, lm2_y), 10, (10,0,250), cv2.FILLED)
            cv2.circle(image, (lm3_x, lm3_y), 5, (30,0,250), cv2.FILLED)
        return int(angle)  
        
    def lm_names(self):
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                # print('ID', id, 'LM', lm)
                vis =  lm.visibility * 100
                name = self.mpPose.PoseLandmark(id).name
                return name


    
    def detect_landmarks(self,img):
        
        img =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # MAKE DETECTIONS
        self.results = self.pose.process(img)
        
        # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        if self.results.pose_landmarks:
            return self.results.pose_landmarks.landmark
    def getPosition(self, img, draw=True):
        self.lmList = []
        
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255,0,0), cv2.FILLED)
        return self.lmList
        
    def findPose(self, img, draw=True):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # MAKE DETECTIONS
        self.results = self.pose.process(img)

        img =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS )
        return img

                
            
        
        
        
        
        
        

## visibility

# Squat workout Pipeline

In [4]:
squatVid = 'squat/squatv4.mp4'
squatStartImg = 'squat/start.png'
squatMidImg = 'squat/mid.png'

In [6]:
detector = WorkoutDetector()

def squat():
    cap = cv2.VideoCapture(squatVid)
    pTime = 0
    count = 0
    stage =None

    while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img, draw=False)

        if len(lmList) > 1:
            angle = int(detector.calculate_angle(image,[23,25,27], draw=False))
    #         detector.draw_angle(image,23,25,27)
            ankle_width = detector.get_distance_btw_landmarks(image,27, 28 )
            knee_width = detector.get_distance_btw_landmarks(image,25, 26 )




            # if knee_width > ankle_width:
            if angle:
                if angle >= 120  and stage == 'down':
                    stage = 'up'
                    count +=1
                if angle <= 100:
                    stage = 'down'
            mycount = "C:{},A:{},S:{} ".format(int(count), angle, stage)
            cv2.putText(image, mycount, (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0,0), 3)
            cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
squat()

LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33


## live camara squat

In [7]:

leg_leg_lms = [23,25,27]
leg_right_lms = [24,26,28]

left_arm_lms = [11,13,15]
right_arm_lms = [12,14,16]


def bent(angle, angle_check=45):
    return angle < angle_check


def not_bent(angle, angle_check=150):
    return angle > angle_check and angle < 187





In [4]:
# VIDEO FEED
detector = WorkoutDetector()

cap = cv2.VideoCapture(0)
name = ' '
pTime = 0
count = 0
stage =None
ankle_width = 0
knee_width = 0
vis = None
angle =0

SQUAT_STAGE = None
SQAUT_UP_ANGLE = 170
STAGE = ''
    
SQUAT_DOWN_ANGLE = 90
message = ' '
org = (0, 300)

stage_notes = ''
angle_notes = ''
message_note = ' '

corrections = ''
IN_FRAME =False
SHOW_START_POSE=True


START_POSE= False



while cap.isOpened():
    ret, frame = cap.read()
    image = detector.findPose(frame)
    
    stage_notes = "S:{}".format( SQUAT_STAGE)
    message_note = "M: {} inframe{}".format( message, IN_FRAME)
    
    angle_notes = "C: {} A: {}".format( count , angle)
    
    correction_note = 'do this {}'.format(corrections)
    
    
    
    landmarks = detector.detect_landmarks(image)
    lmList = detector.getPosition(image, draw=False)
    if landmarks:
#         vis_left_ankle = detector.check_lm_visibility(landmarks, 'nose')
#         vis_nose = detector.check_lm_visibility(landmarks, 'right_hip')
        
        if detector.body_in_frame(landmarks):
            IN_FRAME = True
            message = 'in frame'
            
        else:
            IN_FRAME = False
            message = 'not in frame'
            
            
        if IN_FRAME:
            ankle_width = detector.get_distance_btw_landmarks(image,27, 28 )
#             knee_width = detector.get_distance_btw_landmarks(image,25, 26 )
                            
            arm_left_angle = detector.calculate_angle(image,left_arm_lms)
            arm_right_angle = detector.calculate_angle(image,right_arm_lms)
            
            if bent(arm_left_angle, 90) or bent(arm_left_angle, 90) and ankle_width > 90:
                SHOW_START_POSE=False
                START_POSE= True
                corrections = 'nice work'
            elif not ankle_width > 90:
                corrections = 'spread legs'
            else:
                SHOW_START_POSE = True
                START_POSE= False
            
            if START_POSE:
                message = 'GO...'
                angle = int(detector.calculate_angle(image,leg_right_lms, draw=False))
                
                if angle >= 150:
                    if SQUAT_STAGE == 'SQUAT_STAGE_DOWN':
                        count +=1
                        SQUAT_STAGE = 'SQUAT_STAGE_UP'
                    SQUAT_STAGE = 'SQUAT_STAGE_UP'
                elif angle <= 99:
                    SQUAT_STAGE = 'SQUAT_STAGE_DOWN'
                else:
                    corrections = 'go lower'
            
            else:
                message = 'spread legs and bent arms'
                
        else:
            # not in frame 
            pass
    else:
        pass
        
    cv2.putText(image, stage_notes , (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    cv2.putText(image, angle_notes , (0,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    
    cv2.putText(image, message_note , (0,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    
    
    cv2.putText(image, correction_note , (0,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    
    
    
    image = cv2.resize(image, (600, 600))                # Resize image
    cv2.imshow('Mediapipe Feed', image)
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# Bicycle Crunches

## Bicycle crunche Stages
1. START_LEFT_LEG_BENTUP_RIGHT_LEG_STRAIGHT
2. START_RIGHT_LEG_BENTUP_LEFT_LEG_STRAIGHT
3. END_LEFT_STRAIGHT_RIGHT_LEG_BENTUP
4. END_RIGHT_STRAIGHT_LEFT_LEG_STRAIGHT






In [6]:
bicycle_crunche_path = 'bicycyle_crunches/'

end_left_leg_down = 'crunche_end_left_leg_down.png'
end_right_leg_down = 'crunche_end_right_leg_down.png'

start_left_leg_up = 'crunche_start_left_leg_up.png'
start_right_leg_up = 'crunche_start_right_leg_up.png'

# bicycle_crunche_path+start_left_leg_up

In [8]:
def bent(angle, angle_check):
    return angle < angle_check




detector = WorkoutDetector()

while True:
    image = cv2.imread(bicycle_crunche_path+start_right_leg_up)
    image = detector.findPose(image, draw=False)
    lmList = detector.getPosition(image, draw=False)


    cv2.imshow('Image', image)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()

NameError: name 'bicycle_crunche_path' is not defined

In [ ]:

detector = WorkoutDetector()

cap = cv2.VideoCapture('lungs/Lunges.mp4')

distance_of_ankles_start = 0
distance_of_ankles = 0
state = {
           "COUNT": 0,
          "LEFT_STAGE" : '',
          "RIGHT_STAGE": '',
          "IN_FRAME": False,
          "START_POSE": False,
          "SHOW_START_POSE": False,
          "CORRECTIONS": '',
          "MESSAGE": 'lung start',
          "STAGE": '',
          "PRE_STAGE":'lung start'
      }
count = 0

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img)
        detector.getPosition(image)
        
        detector.getPosition(image, draw=False)
        landmarks = detector.detect_landmarks(image)

        if landmarks:
            
            if detector.body_in_frame(landmarks):
                state['IN_FRAME'] = True

                left_leg = detector.calculate_angle(image,leg_leg_lms, draw=False)
                right_leg = detector.calculate_angle(image,leg_right_lms, draw=False)

                arm_left_angle = detector.calculate_angle(image,left_arm_lms, draw=False)
                arm_right_angle = detector.calculate_angle(image,right_arm_lms,  draw=False)
                dis = detector.get_distance_btw_landmarks(image,27, 28,  draw=False)
                if bent(arm_left_angle, 90) or bent(arm_right_angle,90):

                    if not_bent(left_leg, 140) and not_bent(right_leg, 140) and dis > 0 and dis < 100:
                        state['PRE_STAGE'] =  state['STAGE']
                        # state['STAGE'] = 'START_STAGE'
                        state['CORRECTIONS'] = 'GO... step one leg backwards'
                        
                    if bent(left_leg, 100) and bent(right_leg, 100):

                        if dis < 0:
                            state['PRE_STAGE'] =  state['STAGE']
                            state['STAGE'] = 'LEFT_LEG_DOWN_START'
                            state['CORRECTIONS'] = 'Now push down on left leg to come up'

                        elif dis > 120:

                            state['PRE_STAGE'] =  state['STAGE']
                            state['STAGE'] = 'RIGHT_LEG_DOWN_START'
                            state['CORRECTIONS'] = 'Now push down on right leg to come up'
                    if not_bent(left_leg, 140) and not_bent(right_leg, 140):

                        if state['STAGE'] == 'LEFT_LEG_DOWN_START':

                            state['COUNT'] += 1
                            state['STAGE'] = 'START_STAGE'
                            print("message count from the {} {}".format(state['STAGE'], state['COUNT']))

                        elif state['STAGE'] == 'RIGHT_LEG_DOWN_START':
                            state['COUNT'] += 1
                            state['STAGE'] = 'START_STAGE'
                            print("message count from the {} {}".format(state['STAGE'], state['COUNT']))
                else:
                    state['CORRECTIONS'] = 'bend arms'
                    state['IN_FRAME'] = False
#             message = '{} {}'.format(state['COUNT'], right_leg)
#             message2 = '{} {} {}'.format(state['STAGE'], distance_of_ankles, state['CORRECTIONS'] )
            
#             cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
#             cv2.putText(image, message2 , (40,200), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
            
            image = cv2.resize(image, (360, 700))                # Resize image
            cv2.imshow('Image', image)
            if cv2.waitKey(10) & 0xff == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()
                
                    
                    

LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33


## In out Jumps

In [10]:
jumps_path =  'in_out_jumps/'

In_and_Out_Jump_mp4 = 'In_and_Out_Jump.mp4'



metest = 'metest.mp4'
metest2 = 'metest2.mp4'

in__out_jump_start = 'in__out_jump_start.png'


close_leg_squat_down = 'in_out_jump_close_leg_squat_down.png'

jump_legs_straight = 'in_out_jump_jump_legs_straight.png'

legs_wide_squat_start = 'in_out_jumps_legs_wide_squat_start.png'

leg_wide_squat_down = 'in_out_jump_leg_wide_squat_down.png'


In [11]:
leg_leg_lms = [23,25,27]
leg_right_lms = [24,26,28]

left_arm_lms = [11,13,15]
right_arm_lms = [12,14,16]


def bent(angle, angle_check=45):
    return angle < angle_check


def not_bent(angle, angle_check=150):
    return angle > angle_check and angle < 187

In [14]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(jumps_path+metest2)

distance_of_ankles_start = 0
distance_of_ankles = 0
stage = None
count = 0

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img)
        
        
        landmarks = detector.detect_landmarks(img)
        detector.getPosition(img)    
        if landmarks:
            vis_nose = detector.check_lm_visibility(landmarks, 'nose')
            vis_right_ankle = detector.check_lm_visibility(landmarks, 'right_knee')
            vis_left_ankle = detector.check_lm_visibility(landmarks, 'left_knee')
            
            if vis_nose and vis_right_ankle or vis_left_ankle:
                left_leg = detector.calculate_angle(image,leg_leg_lms)
                right_leg = detector.calculate_angle(image,leg_right_lms)

                arm_left_angle = detector.calculate_angle(image,left_arm_lms)
                arm_right_angle = detector.calculate_angle(image,right_arm_lms)
                
                message = 'dis {} stage {} legs {} {} count {}'.format(distance_of_ankles, stage, left_leg, right_leg, count)
                color = (255,0, 0)
                if bent(arm_left_angle) and bent(arm_left_angle):
                    distance_of_ankles = detector. get_distance_btw_landmarks(image,27, 28)

                    if not_bent(left_leg) and not_bent(right_leg) and distance_of_ankles < 100:
                        if stage == 'STAGE_DOWN_TWO':
                            count += 1
                            stage = 'STAGE_START'
                        stage = 'STAGE_START'            

                    if bent(left_leg, 150) and bent(right_leg, 150) and stage == 'STAGE_START':
                        stage = 'STAGE_DOWN_ONE'

                    if bent(left_leg, 150) and bent(right_leg, 150) and distance_of_ankles > 100:
                        stage = 'STAGE_DOWN_TWO'
#                 color = (3, 56, 11)
                
                
                
                    cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (300, 700))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [42]:

detector = WorkoutDetector()

cap = cv2.VideoCapture(jumps_path+'metest3.mp4')

distance_of_ankles_start = 0
distance_of_ankles = 0
state = {
          "COUNT": 0,
          "IN_FRAME": False,
          "START_POSE": False,
          "SHOW_START_POSE": False,
          "CORRECTIONS": '',
          "MESSAGE": 'lung start',
          "STAGE": ''
      }
count = 0

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img)
        detector.getPosition(image)
        
        detector.getPosition(image, draw=False)
        landmarks = detector.detect_landmarks(image)

        if landmarks:
            
            if detector.body_in_frame(landmarks):
                state['IN_FRAME'] = True
                left_leg = detector.calculate_angle(image,leg_leg_lms)
                right_leg = detector.calculate_angle(image,leg_right_lms)

                arm_left_angle = detector.calculate_angle(image,left_arm_lms)
                arm_right_angle = detector.calculate_angle(image,right_arm_lms)
                if bent(arm_left_angle, 90) and bent(arm_right_angle, 90):
                    state['CORRECTIONS'] = 'Adjust leg'
                    
                    distance_of_ankles = detector. get_distance_btw_landmarks(image,27, 28)
                    if  distance_of_ankles < 250:

                        state['CORRECTIONS'] = 'Go....'

                        if bent(left_leg, 165) and bent(right_leg, 165):
                            state['CORRECTIONS'] = 'nice'
                            state['STAGE'] = 'STAGE_DOWN_ONE'
                        else:
                            state['CORRECTIONS'] = 'Take your squad lower'

                        if not_bent(left_leg, 165) and not_bent(right_leg, 165):
                            if state['STAGE'] == 'STAGE_DOWN_TWO':
                                state['COUNT'] += 1
                                state['STAGE'] = 'STAGE_START'
                    if bent(left_leg, 165) and bent(right_leg, 165) and distance_of_ankles > 250 and state['STAGE'] == 'STAGE_DOWN_ONE':
                        state['STAGE'] = 'STAGE_DOWN_TWO'

                    elif distance_of_ankles < 250:
                        state['CORRECTIONS'] = 'spread legs wider'
                    else:
                        state['CORRECTIONS'] = 'Take your squad lower'
                        
                else:
                    state['CORRECTIONS'] = 'bend arms'
                    state['IN_FRAME'] = False
            message = '{} {}'.format(state['COUNT'], right_leg)
            message2 = '{} {}'.format(state['STAGE'], distance_of_ankles)
            message3 = '{}'.format(state['CORRECTIONS'] )
            
            cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
            cv2.putText(image, message2 , (40,200), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
            cv2.putText(image, message3 , (60,250), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
            
            
            image = cv2.resize(image, (360, 700))                # Resize image
            cv2.imshow('Image', image)
            if cv2.waitKey(10) & 0xff == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()
                
                    
                    

LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33


LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33
LENGHT 33


## Donkey Kicks

In [10]:
path_donkey_kicks = 'donkey_kicks/'
Donkey_Kicks = 'Donkey_Kicks.mp4'

In [29]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(path_donkey_kicks+Donkey_Kicks)

left_stage = None
right_stage = None

count = 0
left_count = 0
right_count = 0
message_note = ' '
stage = ''

corrections = ''
IN_FRAME =False
SHOW_START_POSE=True

color = (255,0, 0)

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        landmarks = detector.detect_landmarks(img)
        detector.getPosition(img)
        
        
        if landmarks:

            if detector.body_in_frame(landmarks):
                IN_FRAME = True
                message = 'in frame'
            
            else:
                IN_FRAME = False
                message = 'not in frame'
            
            
            if IN_FRAME:
                
                vis_nose = detector.check_lm_visibility(landmarks, 'nose')
                vis_right_ankle = detector.check_lm_visibility(landmarks, 'right_ankle')
                vis_left_ankle = detector.check_lm_visibility(landmarks, 'left_ankle')


                left_hip_shoulder_wrist_angle = detector.calculate_angle(image,[23,11,15])
                right_hip_shouldleft_hip_shoulder_wrist_angleer_wrist_angle = detector.calculate_angle(image,[24,12,16])
                
                left_shoulder_hip_knee_angle = detector.calculate_angle(image,[11,23,25])

                right_shoulder_hip_knee_angle = detector.calculate_angle(image,[12,24,26])


                left_leg = detector.calculate_angle(image,leg_leg_lms, draw=False)
                right_leg = detector.calculate_angle(image,leg_right_lms, draw=False)
                left_leg_rise = detector.calculate_angle(image,[11,23,25], draw=False)
                right_leg_rise = detector.calculate_angle(image,[12,24,26], draw=False)
                nose_down = detector.calculate_angle(image,[16,12,0])


#                 if bent(left_shoulder_hip_knee_angle, 120) and bent(left_shoulder_hip_knee_angle, 95) or 
#                 bent(right_shoulder_hip_knee_angle, 120) and bent(right_shoulder_hip_knee_angle, 95):
                    
#                     pass
                
                if left_hip_shoulder_wrist_angle < 90 and left_hip_shoulder_wrist_angle < 90:
                    is_true = True
                


                    if bent(left_leg_rise, 125):
                        left_stage = 'LEFT_LEG_DOWN_START'

                    if not_bent(left_leg_rise, 160) and left_stage =='LEFT_LEG_DOWN_START':
                        count +=1
                        left_stage = 'LEFT_LEG_UP'

                    if not_bent(right_leg_rise, 160) and right_stage =='RIGHT_LEG_DOWN_START':
                        count +=1
                        right_stage = 'RIGHT_LEG_UP'


                    if bent(right_leg_rise, 125):
                        right_stage = 'RIGHT_LEG_DOWN_START'

                msg = 'stage {}  leg_rise {}  count {} {} '.format(stage, left_leg_rise, nose_down, count  )


                cv2.putText(image, msg , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [16]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(path_donkey_kicks+Donkey_Kicks)

left_stage = None
right_stage = None

count = 0
left_count = 0
right_count = 0
message_note = ' '

corrections = ''
IN_FRAME =False
SHOW_START_POSE=True

color = (255,0, 0)

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        landmarks = detector.detect_landmarks(img)
        detector.getPosition(img)
        
        
        if landmarks:
            left_hip_shoulder_wrist_angle = detector.calculate_angle(image,[23,11,15])
            right_hip_shoulder_wrist_angle = detector.calculate_angle(image,[24,12,16])


            left_shoulder_hip_knee_angle = detector.calculate_angle(image,[11,23,25])
            right_shoulder_hip_knee_angle = detector.calculate_angle(image,[12,24,26])


            if detector.body_in_frame(landmarks):
                IN_FRAME = True
                message = 'in frame'
            
            else:
                IN_FRAME = False
                message = 'not in frame'
            
            
            if IN_FRAME:
                
                vis_nose = detector.check_lm_visibility(landmarks, 'nose')
                vis_right_ankle = detector.check_lm_visibility(landmarks, 'right_ankle')
                vis_left_ankle = detector.check_lm_visibility(landmarks, 'left_ankle')

                if vis_nose and vis_right_ankle or vis_left_ankle:
                    is_true = True

                left_leg = detector.calculate_angle(image,leg_leg_lms, draw=False)
                right_leg = detector.calculate_angle(image,leg_right_lms, draw=False)
                left_leg_rise = detector.calculate_angle(image,[11,23,25], draw=False)
                right_leg_rise = detector.calculate_angle(image,[12,24,26], draw=False)
                
#                 if bent(left_shoulder_hip_knee_angle, 120) and bent(left_shoulder_hip_knee_angle, 95) or 
#                 bent(right_shoulder_hip_knee_angle, 120) and bent(right_shoulder_hip_knee_angle, 95):
                    
#                     pass
                

                if left_hip_shoulder_wrist_angle < 120 and left_shoulder_hip_knee_angle < 120:
                    stage = 'START STAGE'
                    
                if bent(left_leg_rise, 125):
                    left_stage = 'LEFT_LEG_DOWN_START'
                    stage = 'LEFT_LEG_DOWN_START'


                if not_bent(left_leg_rise, 160) and left_stage =='LEFT_LEG_DOWN_START':
                    count +=1
                    left_stage = 'LEFT_LEG_UP'
                    stage = 'LEFT_LEG_UP'


                if not_bent(right_leg_rise, 160) and right_stage =='RIGHT_LEG_DOWN_START':
                    count +=1
                    right_stage = 'RIGHT_LEG_UP'
                    stage = 'RIGHT_LEG_UP'


                if bent(right_leg_rise, 125):
                    right_stage = 'RIGHT_LEG_DOWN_START'
                    stage = 'RIGHT_LEG_DOWN_START'

                msg = 'stage {}  leg_rise {}  count {} {} {}'.format(stage, left_leg_rise, count, left_hip_shoulder_wrist_angle, left_shoulder_hip_knee_angle  )


                cv2.putText(image, msg , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [17]:
detector = WorkoutDetector()



while True:
        image = cv2.imread(jumps_path+jump_legs_straight)
        image = detector.findPose(image, draw=False)
        
        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img)
        
        left_leg = detector.calculate_angle(image,leg_leg_lms, draw=False)
        right_leg = detector.calculate_angle(image,leg_right_lms, draw=False)
        left_leg_rise = detector.calculate_angle(image,[11,23,25])
        right_leg_rise = detector.calculate_angle(image,[12,24,26])
        
        message = 'legs {} {} leg_rise {} {} '.format(left_leg, right_leg, left_leg_rise, right_leg_rise)
        
        
        

                
                
        cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

NameError: name 'jumps_path' is not defined

In [7]:
path_lunge = 'lungs/'
lunge_down = 'lunge_down.png'

lungeVid = 'Lunges.mp4'
lunges_start = 'lunges_start.png'

In [9]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(path_lunge+lungeVid)

left_stage = None
right_stage = None
stage = ''
count = 0
left_count = 0
right_count = 0
dis = 0 

color = (255,0, 0)

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img, draw=False)
        
        left_leg = detector.calculate_angle(image,leg_leg_lms, draw=True)
        right_leg = detector.calculate_angle(image,leg_right_lms, draw=True)
        
        arm_left_angle = detector.calculate_angle(image,left_arm_lms)
        arm_right_angle = detector.calculate_angle(image,right_arm_lms)
        dis = detector.get_distance_btw_landmarks(image,27, 28)
        
        message = 'dis {} angles {} > {} count {} stage {}'.format(dis, arm_left_angle, arm_right_angle, count, stage )
        
        
        
        if not_bent(left_leg, 140) and not_bent(right_leg, 140):
            if stage == 'RIGHT_LEG_DOWN_START':
                stage = 'START_STAGE'
                count += 1
                
            if stage == 'LEFT_LEG_DOWN_START':
                stage = 'START_STAGE'
                count += 1
        
        if bent(arm_left_angle, 90) and bent(arm_right_angle,90):
            if dis > 0 and dis < 10:
                stage = 'START_STAGE'
                
            if bent(left_leg, 100) and bent(right_leg, 100):

                if dis < 0:
                    stage = 'RIGHT_LEG_DOWN_START'
                elif dis > 150:
                    stage = 'LEFT_LEG_DOWN_START'

        
        
        cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()




In [26]:
-200  0

0